# $Yvette$ $WorkBook$

# Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats
from sklearn.cluster import KMeans

import wrangle as w

import model as m
import env

In [2]:
df = w.get_zillow_data()

In [3]:
df.log_error.describe()

count    52319.000000
mean         0.018136
std          0.176904
min         -4.655420
25%         -0.024702
50%          0.006935
75%          0.040599
max          5.262999
Name: log_error, dtype: float64

In [4]:
df= w.zillow_prep(df)

In [6]:
df.shape

(50293, 35)

In [7]:
train,validate,test, X_train, X_validate, X_test, y_train, y_validate, y_test = w.split(df, 'log_error')

In [8]:
train, test, validate, train_scaled, validate_scaled, test_scaled = m.create_cluster('cluster_loc', train,validate, test,['longitude', 'latitude','tax_value'],4)


In [9]:
train

,parcelid,bedrooms,bathrooms,calculatedbathnbr,fullbathcnt,age,yearbuilt,has_basement,has_deck,has_fireplace,...,tax_value,tax_value_land,taxamount,log_error,abs_log_error,cluster_loc,cluster_loc_0,cluster_loc_1,cluster_loc_2,cluster_loc_3
46594,12741442,3.0,2.0,2.0,2.0,61.0,1956.0,0,0,0,...,327117.0,219824.0,4224.73,0.037870,0.037870,1,0,1,0,0
5692,11283097,4.0,2.0,2.0,2.0,29.0,1988.0,0,0,0,...,198400.0,49700.0,3225.07,0.122093,0.122093,3,0,0,0,1
41034,14262082,3.0,1.5,1.5,1.0,59.0,1958.0,0,0,0,...,130820.0,82481.0,1726.24,0.075170,0.075170,1,0,1,0,0
17083,12050297,2.0,1.0,1.0,1.0,91.0,1926.0,0,0,0,...,68089.0,40359.0,770.05,-0.050191,0.050191,3,0,0,0,1
31106,12689873,4.0,5.0,5.0,5.0,27.0,1990.0,0,0,0,...,1914031.0,1429595.0,22330.56,0.077013,0.077013,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45242,12718148,3.0,1.0,1.0,1.0,64.0,1953.0,0,0,0,...,70229.0,30038.0,1363.71,0.026142,0.026142,1,0,1,0,0
11882,12040330,3.0,1.0,1.0,1.0,69.0,1948.0,0,0,0,...,573005.0,458405.0,6225.34,-0.030627,0.030627,3,0,0,0,1
3384,10858066,2.0,1.0,1.0,1.0,69.0,1948.0,0,0,0,...,140316.0,98225.0,1826.38,0.208652,0.208652,3,0,0,0,1
28632,10866967,6.0,7.0,7.0,7.0,58.0,1959.0,0,0,0,...,1689475.0,1015877.0,20564.61,0.088304,0.088304,0,1,0,0,0


In [13]:
train, test, validate, train_scaled, validate_scaled, test_scaled = m.create_cluster('cluster_price_size', train,validate, test,['taxamount', 'sqft', 'lot_sqft'],4)


In [14]:
train, test, validate, train_scaled, validate_scaled, test_scaled = m.create_cluster('cluster_delinquency_value', train,validate, test,['tax_value', 'sqft', 'lot_sqft','has_taxdelinquency'],4)


In [15]:
train

,parcelid,bedrooms,bathrooms,calculatedbathnbr,fullbathcnt,age,yearbuilt,has_basement,has_deck,has_fireplace,...,cluster_price_size,cluster_price_size_0,cluster_price_size_1,cluster_price_size_2,cluster_price_size_3,cluster_delinquency_value,cluster_delinquency_value_0,cluster_delinquency_value_1,cluster_delinquency_value_2,cluster_delinquency_value_3
46594,12741442,3.0,2.0,2.0,2.0,61.0,1956.0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
5692,11283097,4.0,2.0,2.0,2.0,29.0,1988.0,0,0,0,...,2,0,0,1,0,0,1,0,0,0
41034,14262082,3.0,1.5,1.5,1.0,59.0,1958.0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
17083,12050297,2.0,1.0,1.0,1.0,91.0,1926.0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
31106,12689873,4.0,5.0,5.0,5.0,27.0,1990.0,0,0,0,...,0,1,0,0,0,2,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45242,12718148,3.0,1.0,1.0,1.0,64.0,1953.0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
11882,12040330,3.0,1.0,1.0,1.0,69.0,1948.0,0,0,0,...,1,0,1,0,0,1,0,1,0,0
3384,10858066,2.0,1.0,1.0,1.0,69.0,1948.0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
28632,10866967,6.0,7.0,7.0,7.0,58.0,1959.0,0,0,0,...,0,1,0,0,0,2,0,0,1,0


## Question 1: Is threre any difference in log_error in homes with less than or equal to 5000sqft and homes above 5000sqft

In [ ]:
sns.scatterplot(y='sqft', x='log_error',
                data=train[train.sqft<= 5000], color='orange')

sns.scatterplot(y='sqft', x='log_error',
                data=train[train.sqft> 5000], 
                color='purple')

plt.title("Does squarefeet make a diffirence? ")
plt.show()

In [ ]:
sns.scatterplot(x='sqft', y='abs_log_error',
                data=train, hue= train.cluster_price_size)




plt.title("Does squarefeet make a diffirence? ")
plt.show()

In [ ]:

#running stats test for these-first checking variance
stat, p = levene(loc_zero.log_error, loc_one.log_error, loc_two.log_error, loc_three.log_error)
stat, p


### clusters 1,2,3 have more abs log error than cluster 0

In [ ]:
sns.scatterplot(y='sqft', x='abs_log_error',
                data=train, hue= train.cluster_size_optionalfea)




plt.title("Does squarefeet make a diffirence? ")
plt.show()

### cluster 2 has more log error and maybe cluster 3

**Properties with less than 5,000 sqft seem to have more log error than houses above 5,000 sqft.**

In [ ]:
# create bins
train['sqft_bin']= pd.cut(train.sqft,[0,5000,19000],labels = ['lessthan_5000sqft', 'morethan_5000sqft'])

In [ ]:
# box plot of the two groups and spending score
sns.boxplot(x=train.sqft_bin, y=train.log_error)
plt.show()

In [ ]:
sns.barplot(x= train.sqft_bin, y= train.abs_log_error);

Homes with less than 5000 sqft seem to have more log error.

In [ ]:
train.sqft_bin.value_counts()

In [ ]:
stat, p_value = stats.levene(train[train.sqft_bin=='lessthan_5000sqft'].log_error, 
                             train[train.sqft_bin=='morethan_5000sqft'].log_error,)
stat, p_value

$H_0$: Log_error of properties with less than 5000sqft >= Log_error mean of properties with more than 5000sqft

$H_a$: Log_error of properties with less than 5000sqft < Log_error mean of properties with more than 5000sqft

# should the test be run against log_erro or absolute log error?

In [ ]:
sample_less = train[train.sqft_bin=='lessthan_5000sqft']
sample_more = train[train.sqft_bin=='morethan_5000sqft']

alpha = 0.05
t, p = stats.ttest_ind(sample_less.abs_log_error,sample_more.abs_log_error,equal_var = True, random_state=123)

print(t, p/2)

**Since p/2 is less than 0.05 then we reject null hypothesis. So there is evidence to suggest that the log_error of properties with less than 5000 sqft is more than the log_error than propeties with more than 5000sqft.**

##################################redudant#######################################

# Custering using price and size

### scale 1st

In [ ]:
# the variables that still need scaling
columns_to_scale = ['bedrooms', 'bathrooms', 'age', 'sqft', 'lot_sqft', 'tax_value_bldg', 'tax_value', 'tax_value_land',
       'taxamount','yearbuilt','has_taxdelinquency']

In [ ]:
train_scaled, X_validate_scaled,X_test_scaled = w.scale_data(train,X_validate, X_test, columns_to_scale, MinMaxScaler())

In [ ]:
train_scaled

# Taxamount _____what i am clustering on price and size

In [ ]:
# list of variables I will cluster on. 
cluster_vars = ['taxamount', 'sqft', 'lot_sqft']
cluster_name = 'price_and_size_cluster'


In [ ]:
X_price_and_size_cluster = train_scaled[['taxamount', 'sqft', 'lot_sqft']]
kmeans = KMeans(n_clusters=4)
kmeans.fit(X_price_and_size_cluster)
train_scaled['cluster'] = kmeans.predict(X_price_and_size_cluster)
train['cluster']=kmeans.predict(X_price_and_size_cluster)
centroids= pd.DataFrame(kmeans.cluster_centers_, columns=X_price_and_size_cluster.columns)

centroids

In [ ]:
train_scaled.cluster

In [ ]:
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(9, 6))
    pd.Series({k: KMeans(k).fit(X_price_and_size_cluster).inertia_ for k in range(2, 12)}).plot(marker='x')
    plt.xticks(range(2, 12))
    plt.xlabel('k')
    plt.ylabel('inertia')
    plt.title('Change in inertia as k increases')

k of interest 3, 4, and 5

In [ ]:
plt.figure(figsize=(14, 9))

for cluster, subset in train_scaled.groupby('cluster'):
    plt.scatter(subset.log_error, subset.lot_sqft, label='cluster' + str(cluster), alpha=.6)

#centroids.plot.scatter(x='taxamount', y='lot_sqft', c='black', marker='x', s=1000, ax=plt.gca(), label='centroid')

plt.legend()
plt.xlabel('taxamount')
plt.ylabel('sqft')
plt.title('Visualizing Cluster Centers')

In [ ]:
train_scaled.log_error

In [ ]:
plt.figure(figsize=(14, 9))

for cluster, subset in train_scaled.groupby('cluster'):
    plt.scatter(x=subset.taxamount, y=subset.log_error, label='cluster' + str(cluster), alpha=.4)

#centroids.plot.scatter(x='taxamount', y='log_error', c='black', marker='x', s=1000, ax=plt.gca(), label='centroid')

plt.legend()
plt.xlabel('taxamount')
plt.ylabel('log error')
plt.title('Visualizing Cluster Centers')

In [ ]:
train_scaled.cluster.value_counts()


# question? Is there a Relationship between green cluster and log error

In [ ]:
for i in train.columns.to_list():    
    sns.scatterplot(x=train.log_error, y=train[i],hue=train.cluster,palette='Accent')
    plt.show()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(13, 13), sharex=True, sharey=True)

for ax, k in zip(axs.ravel(), range(2, 6)):
    clusters = KMeans(k).fit(X_price_and_size_cluster).predict(X_price_and_size_cluster)
    ax.scatter(X_price_and_size_cluster.taxamount, X_price_and_size_cluster.sqft, c=clusters)
    ax.set(title='k = {}'.format(k), xlabel='taxamount', ylabel='sqft')

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

x = X_price_and_size_cluster.sqft
y = X_price_and_size_cluster.lot_sqft
z = X_price_and_size_cluster.taxamount


fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z,
           linewidths=1, alpha=.7,
           edgecolor='k',
           s = 200,
           c=z)
plt.show()

In [ ]:
train.tax_value

# Tax_value ______what i am clustering on price and size

In [ ]:
train.columns

In [ ]:
# list of variables I will cluster on. 
cluster_vars2 = ['tax_value', 'sqft', 'lot_sqft','has_taxdelinquency']
cluster_name = 'price_and_size_cluster'


In [ ]:
X_tax_value = train_scaled[['tax_value', 'sqft', 'lot_sqft','has_taxdelinquency']]
kmeans = KMeans(n_clusters=4)
kmeans.fit(X_tax_value)
train_scaled['cluster2'] = kmeans.predict(X_tax_value)
train['cluster2'] = kmeans.predict(X_tax_value)
centroids2= pd.DataFrame(kmeans.cluster_centers_, columns=X_tax_value.columns)

centroids2

In [ ]:
train_scaled.cluster2

In [ ]:
with plt.style.context('seaborn-whitegrid'):
    plt.figure(figsize=(9, 6))
    pd.Series({k: KMeans(k).fit(train_scaled[['tax_value', 'sqft', 'lot_sqft','has_taxdelinquency']]).inertia_ for k in range(2, 12)}).plot(marker='x')
    plt.xticks(range(2, 12))
    plt.xlabel('k')
    plt.ylabel('inertia')
    plt.title('Change in inertia as k increases')

k of interest 3, 4, and 5

In [ ]:
plt.figure(figsize=(14, 9))

for cluster2, subset in train_scaled.groupby('cluster2'):
    plt.scatter(subset.tax_value, subset.has_taxdelinquency, label='cluster2' + str(cluster2), alpha=.6)

centroids2.plot.scatter(x='tax_value', y='sqft', c='black', marker='x', s=1000, ax=plt.gca(), label='centroid')

plt.legend()
plt.xlabel('tax_value')
plt.ylabel('sqft')
plt.title('Visualizing Cluster Centers')

In [ ]:
sns.histplot(train.log_error,bins=120)

In [ ]:
for i in train.columns.to_list():    
    sns.scatterplot(x=train.log_error, y=train[i],hue=train.cluster2,palette='Accent')
    plt.show()

In [ ]:
sns.scatterplot(x=train.log_error, y=train.sqft,hue=train.cluster2,palette='Accent')

In [ ]:
train_scaled.cluster.value_counts()


In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(13, 13), sharex=True, sharey=True)

for ax, k in zip(axs.ravel(), range(2, 12)):
    clusters = KMeans(k).fit(train_scaled[['tax_value', 'sqft', 'lot_sqft','has_taxdelinquency']]).predict(train_scaled[['tax_value', 'sqft', 'lot_sqft','has_taxdelinquency']])
    ax.scatter(train_scaled['tax_value'], train_scaled['sqft'], c=clusters)
    ax.set(title='k = {}'.format(k), xlabel='taxamount', ylabel='sqft')

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

x = X_price_and_size_cluster.sqft
y = X_price_and_size_cluster.lot_sqft
z = X_price_and_size_cluster.taxamount


fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z,
           linewidths=1, alpha=.7,
           edgecolor='k',
           s = 200,
           c=z)
plt.show()

#######################*********************************END Redundant ************************************###########################

# Question 2: Is there more log_error in Older homes?


In [ ]:
train.yearbuilt.describe()

In [ ]:
sns.histplot(train.yearbuilt)

In [ ]:
sns.scatterplot(y='yearbuilt', x='log_error',
                data=train[train.yearbuilt<= 1935], color='orange')

sns.scatterplot(y='yearbuilt', x='log_error',
                data=train[(train.yearbuilt> 1935) & (train.yearbuilt<=1950)], 
                color='purple')
sns.scatterplot(y='yearbuilt', x='log_error',
                data=train[(train.yearbuilt> 1950) & (train.yearbuilt<1979)], 
                color='blue')
sns.scatterplot(y='yearbuilt', x='log_error',
                data=train[train.yearbuilt>1979], 
                color='brown')

plt.title("Does yearbuilt  make a diffirence? ")
plt.show()

**Yearbuilt does not seem to be an indicator of log_error with current bins. There is error in all ages more so in the years 1940 and above. Maybe if bins where distributed diffirently could lead to new discovery**

In [ ]:
sns.scatterplot(y='yearbuilt', x='log_error',
                data=train[train.yearbuilt<= 1935], color='orange')

sns.scatterplot(y='yearbuilt', x='log_error',
                data=train[train.yearbuilt> 1935], 
                color='purple')


plt.title("Does yearbuilt  make a diffirence? ")
plt.show()

In [ ]:
train['yearbuilt_bins'] = pd.cut(train.yearbuilt, [0,1935,2020],labels = ['homes_lessthan_1935', 'homes_morethan_1935'])


In [ ]:
train.yearbuilt_bins.value_counts()

In [ ]:
sns.scatterplot(data= train, x='log_error', y='yearbuilt', hue=train.cluster_size_optionalfea, palette='Accent',alpha=.6)

In [ ]:
sns.relplot(data= train, x='abs_log_error', y='yearbuilt', hue=train.cluster_price_size, palette='Accent')

### there seems to be more error in cluster 1

In [ ]:
sns.relplot(data= train, x='abs_log_error', y='yearbuilt', hue=train.cluster_size_optionalfea, palette='Accent')

In [ ]:
stat, p_value = stats.levene(train[train.yearbuilt_bins=='homes_morethan_1935'].log_error, 
                             train[train.yearbuilt_bins=='homes_lessthan_1935'].log_error)
stat, p_value

$H_0$: Log_error of properties whose yearbuilt is less or equal than 1935 >= Log_error mean of properties whose yearbuilt is more than 1935

$H_a$: Log_error of properties whose yearbuilt is less or equal than 1935 < Log_error mean of properties whose yearbuilt is more than 1935

In [ ]:
sample_more = train[train.yearbuilt_bins=='homes_morethan_1935']
sample_less= train[train.yearbuilt_bins=='homes_lessthan_1935']

alpha = 0.05
t, p = stats.ttest_ind(sample_less.log_error,sample_more.log_error,equal_var = True, random_state=123)

print(t, p/2)

We reject the null hypotheis. There is evidence to suggest that the mean log error of properties whose yearbuilt is below 1935 is mless than log error of homes built after 1935

In [ ]:
sns.scatterplot(y='age', x='log_error',
                data=train[train.age<= 81], color='orange')

sns.scatterplot(y='age', x='log_error',
               data=train[train.age> 81], 
               color='purple')


plt.title("Does yearbuilt  make a diffirence? ")
plt.show()

In [ ]:
train.age.describe()

In [ ]:
train['age_bins'] = pd.cut(train.age, [0,81,150],labels = ['homes_lessthan_81', 'homes_morethan_81'])


In [ ]:
train.age_bins.value_counts()

In [ ]:
stat, p_value = stats.levene(train[train.age_bins=='homes_morethan_81'].log_error, 
                             train[train.age_bins=='homes_lessthan_81'].log_error)
stat, p_value

In [ ]:
sample_more = train[train.age_bins=='homes_morethan_81']
sample_less= train[train.age_bins=='homes_lessthan_81']

alpha = 0.05
t, p = stats.ttest_ind(sample_less.log_error,sample_more.log_error,equal_var = True, random_state=123)

print(t, p/2)

In [ ]:
train.fips

# Question 3 longitute vs latitude log error

In [ ]:
sns.scatterplot(x=train.longitude, y=train.latitude,hue=train.log_error)

# question? Is there a Relationship between green cluster and log error

In [ ]:
sns.histplot(train.cluster_price_size)

'taxamount', 'sqft', 'lot_sqft features in cluster of interest

In [ ]:
sns.scatterplot(x=train.log_error, y=train.sqft)

In [ ]:
plt.figure(figsize=(14, 9))

for cluster, subset in train.groupby('cluster_price_size'):
    plt.scatter(x=subset.taxamount, y=subset.log_error, label='cluster' + str(cluster), alpha=.7, )
plt.legend()
plt.xlabel('taxamount')
plt.ylabel('log error')
plt.title('Visualizing Cluster Centers')

In [ ]:
cluster3_price_and_size = train[train.cluster_price_size==3]

In [ ]:
cluster1_price_and_size = train[train.cluster_price_size==1]

In [ ]:
cluster2_price_and_size = train[train.cluster_price_size==2]

In [ ]:
cluster0_price_and_size = train[train.cluster_price_size==0]

In [ ]:
alpha = 0.05
t, p = stats.ttest_ind(cluster0_price_and_size.log_error,cluster3_price_and_size.log_error,equal_var = True, random_state=123)

print(t, p/2)

In [ ]:
plt.figure(figsize=(14, 9))

for cluster, subset in train.groupby('cluster_price_size3'):
    plt.scatter(x=subset.taxamount, y=subset.log_error, label='cluster' + str(cluster), alpha=.7, )
plt.legend()
plt.xlabel('taxamount')
plt.ylabel('log error')
plt.title('Visualizing Cluster Centers')

In [ ]:
cluster1_price_and_size3 = train[train.cluster_price_size3==1]

In [ ]:
cluster2_price_and_size3 = train[train.cluster_price_size3==2]

In [ ]:
cluster0_price_and_size3 = train[train.cluster_price_size3==0]

In [ ]:
alpha = 0.05
t, p = stats.ttest_ind(cluster2_price_and_size3.log_error,cluster1_price_and_size3.log_error,equal_var = True, random_state=123)

print(t, p/2)

In [ ]:
sns.relplot(x=train.abs_log_error, y= train.tax_value )

In [ ]:
features = ['sqft','taxamount']

# Models

In [ ]:
def get_models_with_results(X_train, y_train, X_validate, y_validate):
    

    # Baseline Model
    # run the model
    pred_mean = y_train.log_error.mean()
    y_train['pred_mean'] = pred_mean
    y_val['pred_mean'] = pred_mean
    rmse_train = mean_squared_error(y_train.value, y_train.pred_mean, squared=False)
    rmse_val = mean_squared_error(y_val.value, y_val.pred_mean, squared=False)

    # save the results
    metrics = pd.DataFrame(data=[{
        'model': 'baseline_mean',
        'rmse_train': rmse_train,
        'r2_train': explained_variance_score(y_train.value, y_train.pred_mean),
        'rmse_val': rmse_val,
        'r2_val': explained_variance_score(y_val.value, y_val.pred_mean)}])

    # LassoLars Model
    # run the model
    lars = LassoLars(alpha=1)
    lars.fit(X_train, y_train.value)
    y_train['pred_lars'] = lars.predict(X_train)
    rmse_train = mean_squared_error(y_train.value, y_train.pred_lars, squared=False)
    y_val['pred_lars'] = lars.predict(X_val)
    rmse_val = mean_squared_error(y_val.value, y_val.pred_lars, squared=False)

    # save the results
    metrics = metrics.append({
        'model': 'LarsLasso, alpha 1',
        'rmse_train': rmse_train,
        'r2_train': explained_variance_score(y_train.value, y_train.pred_lars),
        'rmse_val': rmse_val,
        'r2_val': explained_variance_score(y_val.value, y_val.pred_lars)}, ignore_index=True)

    # Polynomial Models
    # set up the model
    pf = PolynomialFeatures(degree=2)
    X_train_d2 = pf.fit_transform(X_train)
    X_val_d2 = pf.transform(X_val)
    
    # run the model
    lm2 = LinearRegression()
    lm2.fit(X_train_d2, y_train.value)
    y_train['pred_lm2'] = lm2.predict(X_train_d2)
    rmse_train = mean_squared_error(y_train.value, y_train.pred_lm2, squared=False)
    y_val['pred_lm2'] = lm2.predict(X_val_d2)
    rmse_val = mean_squared_error(y_val.value, y_val.pred_lm2, squared=False)

    # save the results
    metrics = metrics.append({
        'model': 'Quadratic Linear Regression',
        'rmse_train': rmse_train,
        'r2_train': explained_variance_score(y_train.value, y_train.pred_lm2),
        'rmse_val': rmse_val,
        'r2_val': explained_variance_score(y_val.value, y_val.pred_lm2)}, ignore_index=True)

    # set up the model
    pf = PolynomialFeatures(degree=3)
    X_train_d2 = pf.fit_transform(X_train)
    X_val_d2 = pf.transform(X_val)

    # run the model
    lm2 = LinearRegression()
    lm2.fit(X_train_d2, y_train.value)
    y_train['pred_lm2'] = lm2.predict(X_train_d2)
    rmse_train = mean_squared_error(y_train.value, y_train.pred_lm2, squared=False)
    y_val['pred_lm2'] = lm2.predict(X_val_d2)
    rmse_val = mean_squared_error(y_val.value, y_val.pred_lm2, squared=False)

    # save the results
    metrics = metrics.append({
        'model': 'Cubic Linear Regression',
        'rmse_train': rmse_train,
        'r2_train': explained_variance_score(y_train.value, y_train.pred_lm2),
        'rmse_val': rmse_val,
        'r2_val': explained_variance_score(y_val.value, y_val.pred_lm2)}, ignore_index=True)

    return metrics
